Windows
D:\git\cardio\venv\Scripts\python.exe
C:\Users\1720267\Anaconda3\python.exe


In [2]:
# imports, setup and personal settings
# use slicer instead of imageJ
#export SITK_SHOW_COMMAND=/Applications/Slicer.app/Contents/MacOS/Slicer
from __future__ import print_function
import logging
import os
import importlib
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('D:\\git\\cardio')
else:
    os.chdir('/Users/Code/Git/cardio')
print(os.getcwd())
import IPython.core.debugger as debugger
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', aspect='equal')
# enable inline plotting
%matplotlib inline
import SimpleITK as sitk

from distutils.version import LooseVersion
import sys
print(sys.version)
print(sys.version_info)

# project related helper functions
from src.utils.file_wrapper.app.Files import Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
from src.utils.file_wrapper.app.Setup import Console_and_file_logger
from src.utils.myshow import myshow, myshow3d
from src.utils.utils_io import __ensure_dir__

# for detailed debug infos
#Console_and_file_logger('ipython_sitk_stack')

# toggle detailed print messages
DETAILED_DEBUG = False

# check that all neccessary packages are installed (see requirements.txt file)
required_packages = {
    'jupyter', 'numpy', 'matplotlib', 'ipywidgets', 'scipy', 'pandas',
    'SimpleITK'
}

problem_packages = list()
# Iterate over the required packages: If the package is not installed
# ignore the exception.
for package in required_packages:
    try:
        p = importlib.import_module(package)
    except ImportError:
        problem_packages.append(package)

if len(problem_packages) is 0:
    print('All is well.')
else:
    print('The following packages are required but not installed: ' \
          + ', '.join(problem_packages))

D:\git\cardio


ModuleNotFoundError: No module named '_SimpleITK'

In [ ]:
class MyDicomfile(Dicomfile):
    """
    super Dicomfile already knows how to open save, desribe and visualize a dcm image
    you need to implement:
    
    file_type
    def __init__(self, dir_name='./', filename='test.dcm', destination='./dest'):
    action(self):

    important, return the file for later statistics
    """
    file_type = '.dcm'

    def __init__(self,
                 dir_name='./',
                 filename='test.dcm',
                 destination='./dest'):
        super(MyDicomfile, self).__init__(dir_name, filename, destination)
        self.stats['filetype'] = self.__class__.file_type

    def action(self):
        logging.info('action performed')
        return self

In [ ]:
src1 = 'data/raw/0001_small/1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857'
src2 = 'data/raw/0001_small/1.3.12.2.1107.5.99.2.1013.30000008030612131778100062911'
dest = 'data/processed/small/test1'


# get all pictures of two studies
fw1 = FileWalker(MyDicomfile, src_dir=src1, dest_dir=dest)
fw2 = FileWalker(MyDicomfile, src_dir=src2, dest_dir=dest)

#print(fw1.open_files[0].stats)

#fw.log_stats()

In [ ]:
#select the first two pictures of each serie
# same timestep, but different slices
img1 = fw1.open_files[0].img
img2 = fw2.open_files[0].img

# show both slices
myshow3d(img1)
myshow3d(img2)

# get the numpy array out of an dcm file
img1_nda = sitk.GetArrayFromImage(img1)
img2_nda = sitk.GetArrayFromImage(img2)

print(img1_nda)
print('numpy shape: {}'.format(img1_nda.shape))
print('sitk shape: {}'.format(img1.GetSize()))

In [ ]:

def stack_with_vstack():
    """
    stack two dcm images with nunpy vstack and show them
    """
    stacked_nda = np.vstack((img1_nda, img2_nda))
    stacked_img = sitk.GetImageFromArray(stacked_nda)
    myshow3d(stacked_img)

stack_with_vstack

In [ ]:

def stack_with_concatenate():
    """
    Concatenate two dcm images with np.concatenate
    """
    stacked_nda_concat = np.concatenate((img1_nda, img2_nda))
    stacked_img_concat = sitk.GetImageFromArray(stacked_nda_concat)
    myshow3d(stacked_img_concat)
    
stack_with_concatenate

In [ ]:

def create_emtpty_sitk():
    """
    Create an empty sitk image and set the size
    """

    merged_img = sitk.Image(192, 168, 2, sitk.sitkUInt16)
    merged_nda = sitk.GetArrayFromImage(merged_img)
    print(merged_nda)
    print('numpy shape: {}'.format(merged_nda.shape))
    print('sitk shape: {}'.format(merged_img.GetSize()))
    
create_emtpty_sitk

In [ ]:
def save(sitk_image, dest, f_name):
    """
    Use simpleITK to write the current dicom file to disk
    :return:
    """
    __ensure_dir__(dest)
    try:
        sitk.WriteImage(sitk_image, os.path.join(dest, f_name))
    except Exception as e:
        print('Failed to save sitk image {} due to: {}'.format(f_name, e.message))


In [ ]:
def create_a_file_walker(src_path, dest_path):
    """
    MyDicomfile wrapper just loads the dcm image
    """
    fw = None
    fw = FileWalker(MyDicomfile, src_dir=src_path, dest_dir=dest_path)
    # logging in notebook must be activated, uncomment the Console_and_file_logger
    fw.log_stats()
    return fw
    

In [ ]:

def get_all_series(path):
    """
    returns a list of all series-Folder
    """
    __ensure_dir__(path)
    series = []
    series = [
        d for d in sorted(os.listdir(path)) if os.path.isdir(os.path.join(path, d))
    ]
    print('Folders in source: {}'.format(len(series)))
    if DETAILED_DEBUG:
        [print(folder + '\n') for folder in series]
    return series

In [ ]:

def create_3d_from_dcm_slices(slices, axis=0):
    slices_nda = []
    volume_nda = []
    
    slices_nda = [sitk.GetArrayFromImage(slice) for slice in slices]
    volume_nda = np.concatenate(slices_nda, axis=axis)
    volume_img = sitk.GetImageFromArray(volume_nda)
    
    if DETAILED_DEBUG:
        print('create 3D image from: {} slices'.format(len(slices)))
        print('Shape of 3D image: {}'.format(volume_nda.shape))
        print('Size of 3D sitk Image {}'.format(volume_img.GetSize()))
        
    return volume_img
        

In [ ]:
def create_histogram(values, labels, title='unnamed', xlabel='', ylabel=''):
    """
    Create a bar chart with nice looking ticks and spacing
    """
    fig, ax = plt.subplots()
    indexes = np.arange(len(labels))
    width = 1 

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xticks(indexes)
    ax.set_xticklabels(labels, rotation=90)
    ax.bar(indexes, values, 0.5, label=labels, align='center')

In [ ]:
def create_statistics(file_walkers):
    """
    get some statistics from the open files
    """
    from collections import Counter
    
    images_per_fw = []
    labels = []

    print('File_walkers: {}'.format(len(file_walkers)))

    # print each serie and the corresponding number of images
    [print('Serie: {} with {} images'.format(fw.open_files[0].dir,len(fw.open_files)))for idx, fw in enumerate(file_walkers)]
    
    # create one list with the stats of all images
    image_stats = []
    for fw in file_walkers:
        for file in fw.open_files:
            image_stats.append(file.stats)
    
    # get the number of open files per file walker
    images_per_fw = [len(fw.open_files) for fw in file_walkers]   
    c = Counter(images_per_fw)
    labels, values = zip(*c.items())   
    create_histogram(values, labels, 'Images per Serie', '# of images', 'total of series')

    # how many open files of which size?
    c1 = Counter(stats['size'] for stats in image_stats)
    labels, values = zip(*c1.items())
    create_histogram(values, labels, 'Distribution of Image-Size', 'Imagesize', 'total in all series')
                 
    
                         
    """
    One example Stats dictionary from the dicom filewarpper
                {
        'filetype':
        '.dcm',
        'file':
        '1.3.12.2.1107.5.99.2.1013.30000008030612131778100062853.dcm',
        'src_path':
        'data/raw/0001-02Z1L14N-2006-11-09/1.3.12.2.1107.5.99.2.1013.30000008030612131778100062857',
        'dimension':
        3,
        'size': (192, 168, 1),
        'origin': (-159.76539802551, -149.47937488556, 68.646492004395),
        'spacing': (1.6145833333333, 1.6145833333333, 6.0)
    }
    """

In [ ]:

def get_all_series_as_file_walker_unfiltered(src, dest):
    """
    Finds all subdirectories for src, create a filewriter per sub-dir
    loads all contained dcm files
    """
    sub_folders = []
    file_walkers = []
    images_per_series = []

    sub_folders = get_all_series(src)
    print('Series in {}: {}'.format(src, len(sub_folders)))
    
    file_walkers = [create_a_file_walker(os.path.join(src, serie_f), dest) for serie_f in sub_folders]
    
    return file_walkers

In [ ]:
def get_origin_from_fw(fw):
    #print(fw.open_files[0].stats['origin'][2])
    return fw.open_files[0].stats['origin'][2]

In [ ]:

def get_all_series_as_file_walker(src, dest):
    """
    Finds all subdirectories for src, create a filewriter per sub-dir
    loads all contained dcm files
    """
    sub_folders = []
    file_walkers = []
    images_per_series = []

    sub_folders = get_all_series(src)
    print('Series in {}: {}'.format(src, len(sub_folders)))
    
    file_walkers = [create_a_file_walker(os.path.join(src, serie_f), dest) for serie_f in sub_folders]

    # filter only the series with 18 included files
    file_walkers_filtered = [fw for fw in file_walkers if len(fw.open_files) is 18]
    print('file walkers filtered')
    
    # sort the file walkers according to the origin of the included images
    file_walkers_sorted = sorted(file_walkers_filtered, key=get_origin_from_fw)
    [print('origin: {}'.format(get_origin_from_fw(fw))) for fw in file_walkers_sorted]
    
    
    return file_walkers_sorted
    

In [ ]:

def stack_to_volumes(file_walkers, dest, persist=False):
    """
    stack all dcm images of the given file_walker
    concatenate fw[n].open_files[j] + fw[n+1].open_files[j] ...
    """

    volume_slices = []
    stacked_volumes = []


    # the len of all open files = timesteps?
    for x in range(len(file_walkers[0].open_files)):
        # concat all slices with timestep n
        # get the n-image from each file walker (series)
        volume_slices = [fw.open_files[x].img for fw in file_walkers]
        # create a volume from the given volume slices
        stacked_volumes.append(create_3d_from_dcm_slices(volume_slices))
        
    # show the stacked volumes
    for idx, volume in enumerate(stacked_volumes):
        myshow3d(volume)
        if persist:
            save(volume, dest, str(idx) + 'volume.dcm')



In [ ]:
# works, but the files are not sorted, volumes are messed up

# src = 'data/raw/0001_small/'
# dest = 'data/processed/stacked/0001_small/'

src = 'data/raw/0001-02Z1L14N-2006-11-09/'
dest = 'data/processed/stacked_sorted/0001-02Z1L14N-2006-11-09/'

f_walkers = get_all_series_as_file_walker(src, dest)

stack_to_volumes(f_walkers, dest, True)


In [ ]:
# Plot some distribution of the data

#src = 'data/raw/0001_small/'
#dest = 'data/processed/stacked/0001_small/'
# 0001-02Z1L14N-2006-11-09/
# 0706-ZY8H5791-2005-06-16/
src = 'data/raw/0001-02Z1L14N-2006-11-09/'
dest = 'data/processed/stacked/0001-02Z1L14N-2006-11-09/'

f_walkers = get_all_series_as_file_walker_unfiltered(src, dest)
create_statistics(f_walkers)



In [3]:
!pip install simpleitk